# LangGraph and LangSmith - Agentic RAG Powered by LangChain

In the following notebook we'll complete the following tasks:

- 🤝 Breakout Room #1:
  1. Install required libraries
  2. Set Environment Variables
  3. Creating our Tool Belt
  4. Creating Our State
  5. Creating and Compiling A Graph!

  - 🤝 Breakout Room #2:
  1. Evaluating the LangGraph Application with LangSmith
  2. Adding Helpfulness Check and "Loop" Limits
  3. LangGraph for the "Patterns" of GenAI

# 🤝 Breakout Room #1

## Part 1: LangGraph - Building Cyclic Applications with LangChain

LangGraph is a tool that leverages LangChain Expression Language to build coordinated multi-actor and stateful applications that includes cyclic behaviour.

### Why Cycles?

In essence, we can think of a cycle in our graph as a more robust and customizable loop. It allows us to keep our application agent-forward while still giving the powerful functionality of traditional loops.

Due to the inclusion of cycles over loops, we can also compose rather complex flows through our graph in a much more readable and natural fashion. Effectively allowing us to recreate application flowcharts in code in an almost 1-to-1 fashion.

### Why LangGraph?

Beyond the agent-forward approach - we can easily compose and combine traditional "DAG" (directed acyclic graph) chains with powerful cyclic behaviour due to the tight integration with LCEL. This means it's a natural extension to LangChain's core offerings!

## Task 1:  Dependencies

We'll first install all our required libraries.

In [1]:
# !pip install -qU langchain langchain-openai langchain-community langgraph arxiv duckduckgo_search==5.3.1b1
!pip install -qU duckduckgo_search


[notice] A new release of pip is available: 23.3.1 -> 24.2
[notice] To update, run: pip install --upgrade pip


## Task 2: Environment Variables

We'll want to set both our OpenAI API key and our LangSmith environment variables.

In [2]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

In [3]:
from uuid import uuid4

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"AIE4 - LangGraph - {uuid4().hex[0:8]}"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("LangSmith API Key: ")

## Task 3: Creating our Tool Belt

As is usually the case, we'll want to equip our agent with a toolbelt to help answer questions and add external knowledge.

There's a tonne of tools in the [LangChain Community Repo](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools) but we'll stick to a couple just so we can observe the cyclic nature of LangGraph in action!

We'll leverage:

- [Duck Duck Go Web Search](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/ddg_search)
- [Arxiv](https://github.com/langchain-ai/langchain/tree/master/libs/community/langchain_community/tools/arxiv)

####🏗️ Activity #1: - `COMPLETED BELOW!!!`

Please add the tools to use into our toolbelt.

> NOTE: Each tool in our toolbelt should be a method.

In [4]:
from langchain_community.tools.ddg_search import DuckDuckGoSearchRun
from langchain_community.tools.arxiv.tool import ArxivQueryRun

tool_belt = [
    ### YOUR CODE HERE☺
    DuckDuckGoSearchRun(),
    ArxivQueryRun()
]

### Model

Now we can set-up our model! We'll leverage the familiar OpenAI model suite for this example - but it's not *necessary* to use with LangGraph. LangGraph supports all models - though you might not find success with smaller models - as such, they recommend you stick with:

- OpenAI's GPT-3.5 and GPT-4
- Anthropic's Claude
- Google's Gemini

> NOTE: Because we're leveraging the OpenAI function calling API - we'll need to use OpenAI *for this specific example* (or any other service that exposes an OpenAI-style function calling API.

In [5]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o", temperature=0)

Now that we have our model set-up, let's "put on the tool belt", which is to say: We'll bind our LangChain formatted tools to the model in an OpenAI function calling format.

In [6]:
model = model.bind_tools(tool_belt)

#### ❓ Question #1:

How does the model determine which tool to use?

## ANSWER #1

The model uses the `description` attribute of each tool.  The `description` is a string that should provide a clear explanation of what the tool does.  This `description` string is passed to the LLM to allow it to decide which tool to use.

For custom user-defined functions to be used as tools, use the @tool decorator and provide a `docstring` for the function to be used as a tool.  The @tool decorator will use the function's `dosctring` as the `description`, so a `docstring` must be provided!

## Task 4: Putting the State in Stateful

Earlier we used this phrasing:

`coordinated multi-actor and stateful applications`

So what does that "stateful" mean?

To put it simply - we want to have some kind of object which we can pass around our application that holds information about what the current situation (state) is. Since our system will be constructed of many parts moving in a coordinated fashion - we want to be able to ensure we have some commonly understood idea of that state.

LangGraph leverages a `StatefulGraph` which uses an `AgentState` object to pass information between the various nodes of the graph.

There are more options than what we'll see below - but this `AgentState` object is one that is stored in a `TypedDict` with the key `messages` and the value is a `Sequence` of `BaseMessages` that will be appended to whenever the state changes.

Let's think about a simple example to help understand exactly what this means (we'll simplify a great deal to try and clearly communicate what state is doing):

1. We initialize our state object:
  - `{"messages" : []}`
2. Our user submits a query to our application.
  - New State: `HumanMessage(#1)`
  - `{"messages" : [HumanMessage(#1)}`
3. We pass our state object to an Agent node which is able to read the current state. It will use the last `HumanMessage` as input. It gets some kind of output which it will add to the state.
  - New State: `AgentMessage(#1, additional_kwargs {"function_call" : "WebSearchTool"})`
  - `{"messages" : [HumanMessage(#1), AgentMessage(#1, ...)]}`
4. We pass our state object to a "conditional node" (more on this later) which reads the last state to determine if we need to use a tool - which it can determine properly because of our provided object!

In [7]:
from typing import TypedDict, Annotated
from langgraph.graph.message import add_messages
import operator
from langchain_core.messages import BaseMessage

class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

## Task 5: It's Graphing Time!

Now that we have state, and we have tools, and we have an LLM - we can finally start making our graph!

Let's take a second to refresh ourselves about what a graph is in this context.

Graphs, also called networks in some circles, are a collection of connected objects.

The objects in question are typically called nodes, or vertices, and the connections are called edges.

Let's look at a simple graph.

![image](https://i.imgur.com/2NFLnIc.png)

Here, we're using the coloured circles to represent the nodes and the yellow lines to represent the edges. In this case, we're looking at a fully connected graph - where each node is connected by an edge to each other node.

If we were to think about nodes in the context of LangGraph - we would think of a function, or an LCEL runnable.

If we were to think about edges in the context of LangGraph - we might think of them as "paths to take" or "where to pass our state object next".

Let's create some nodes and expand on our diagram.

> NOTE: Due to the tight integration with LCEL - we can comfortably create our nodes in an async fashion!

In [8]:
from langgraph.prebuilt import ToolNode

def call_model(state):
  messages = state["messages"]
  response = model.invoke(messages)
  return {"messages" : [response]}

tool_node = ToolNode(tool_belt)

Now we have two total nodes. We have:

- `call_model` is a node that will...well...call the model
- `tool_node` is a node which can call a tool

Let's start adding nodes! We'll update our diagram along the way to keep track of what this looks like!


In [9]:
from langgraph.graph import StateGraph, END

uncompiled_graph = StateGraph(AgentState)

uncompiled_graph.add_node("agent", call_model)
uncompiled_graph.add_node("action", tool_node)

Let's look at what we have so far:

![image](https://i.imgur.com/md7inqG.png)

Next, we'll add our entrypoint. All our entrypoint does is indicate which node is called first.

In [10]:
uncompiled_graph.set_entry_point("agent")

![image](https://i.imgur.com/wNixpJe.png)

Now we want to build a "conditional edge" which will use the output state of a node to determine which path to follow.

We can help conceptualize this by thinking of our conditional edge as a conditional in a flowchart!

Notice how our function simply checks if there is a "function_call" kwarg present.

Then we create an edge where the origin node is our agent node and our destination node is *either* the action node or the END (finish the graph).

It's important to highlight that the dictionary passed in as the third parameter (the mapping) should be created with the possible outputs of our conditional function in mind. In this case `should_continue` outputs either `"end"` or `"continue"` which are subsequently mapped to the action node or the END node.

In [11]:
def should_continue(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  return END

uncompiled_graph.add_conditional_edges(
    "agent",
    should_continue
)

Let's visualize what this looks like.

![image](https://i.imgur.com/8ZNwKI5.png)

Finally, we can add our last edge which will connect our action node to our agent node. This is because we *always* want our action node (which is used to call our tools) to return its output to our agent!

In [12]:
uncompiled_graph.add_edge("action", "agent")

Let's look at the final visualization.

![image](https://i.imgur.com/NWO7usO.png)

All that's left to do now is to compile our workflow - and we're off!

In [13]:
compiled_graph = uncompiled_graph.compile()

#### ❓ Question #2:

Is there any specific limit to how many times we can cycle?

If not, how could we impose a limit to the number of cycles?

## ANSWER #2:

In the way the graph is set up above, there is no specific limit to the number of times we can cycle through the nodes.  As it is currently written, the only time the code will end is if the AIMessage that is appended to the AgentState in the agent node does not have a `tool_calls` kwarg set.  And as it is currently written this implies a non-deterministic flow through the graph.

We can impose a limit to number of cycles by augmenting the definition of the state to include a counter on the number of times that the code has cycled through the graph.  For example, the counter can increment each time the `agent` node is traversed.  If the number of loops exceeds a maximum count, which can be set as a parameter, then we can include this additional exit condition in the should_continue function.

## Using Our Graph

Now that we've created and compiled our graph - we can call it *just as we'd call any other* `Runnable`!

Let's try out a few examples to see how it fairs:

In [14]:
from langchain_core.messages import HumanMessage

inputs = {"messages" : [HumanMessage(content="Who is the current captain of the Winnipeg Jets?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_tolps3eP5QyhRd8dZpdg7SYs', 'function': {'arguments': '{"query":"current captain of the Winnipeg Jets 2023"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 25, 'prompt_tokens': 156, 'total_tokens': 181}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-206c3410-d82c-46cf-89b0-b13da099791c-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current captain of the Winnipeg Jets 2023'}, 'id': 'call_tolps3eP5QyhRd8dZpdg7SYs', 'type': 'tool_call'}], usage_metadata={'input_tokens': 156, 'output_tokens': 25, 'total_tokens': 181})]



Receiving update from node: 'action'
[ToolMessage(content='September 12, 2023. There are not many honours in team sports bigger than being named captain. That honour was given to Winnip

## MY TRIAL #1: Captains of All Canadian NHL Teams

To experiment with multiple and possibly parallel calls to the Tools in ToolBelt

In [15]:
user_query = "Who is the current captain of the team that plays ice hockey in the NHL in Canada?"

In [17]:
inputs = {"messages" : [HumanMessage(content=user_query)]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

print(chunk['agent']['messages'][0].content)

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_mi4RDbVVvGoHOPZ5W1uHi9lP', 'function': {'arguments': '{"query":"current captain of NHL teams in Canada 2023"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 164, 'total_tokens': 190}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-35dd1364-a1ad-4152-9daa-8f8836f1cfc9-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current captain of NHL teams in Canada 2023'}, 'id': 'call_mi4RDbVVvGoHOPZ5W1uHi9lP', 'type': 'tool_call'}], usage_metadata={'input_tokens': 164, 'output_tokens': 26, 'total_tokens': 190})]



Receiving update from node: 'action'
[ToolMessage(content='The 2023-24 NHL season is finally here, and for some players, they\'ll be entering the year with the honored symbol of lea

## MY TRIAL #2: Are Any of the Captains Co-authors of a Paper on Arxiv?

In [18]:
inputs = {"messages" : [HumanMessage(content="Who is the current captain of the team that plays ice hockey in the NHL in Canada?And, is there any paper on Arxiv that mentions one of the captains?")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

print(chunk['agent']['messages'][0].content)

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_Lld11qQATC5PKL4eUbBEw8zi', 'function': {'arguments': '{"query": "current captains of NHL teams in Canada 2023"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_2ySl5aV58PA9G0iil5Yoetqw', 'function': {'arguments': '{"query": "NHL captain"}', 'name': 'arxiv'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 59, 'prompt_tokens': 181, 'total_tokens': 240}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-d4b898f9-bc7e-420a-828f-7ed2b635868d-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'current captains of NHL teams in Canada 2023'}, 'id': 'call_Lld11qQATC5PKL4eUbBEw8zi', 'type': 'tool_call'}, {'name': 'arxiv', 'args': {'query': 'NHL captain'}, 'id': 'call_2ySl5aV58PA9G0iil5Yoetqw', 'type': 'tool_call'}], usage_metadata={

Let's look at what happened:

1. Our state object was populated with our request
2. The state object was passed into our entry point (agent node) and the agent node added an `AIMessage` to the state object and passed it along the conditional edge
3. The conditional edge received the state object, found the "tool_calls" `additional_kwarg`, and sent the state object to the action node
4. The action node added the response from the OpenAI function calling endpoint to the state object and passed it along the edge to the agent node
5. The agent node added a response to the state object and passed it along the conditional edge
6. The conditional edge received the state object, could not find the "tool_calls" `additional_kwarg` and passed the state object to END where we see it output in the cell above!

Now let's look at an example that shows a multiple tool usage - all with the same flow!

In [19]:
import pprint

In [22]:
inputs = {"messages" : [HumanMessage(content="Search Arxiv for the QLoRA paper, then search each of the authors to find out their latest Tweet using DuckDuckGo.")]}

async for chunk in compiled_graph.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        if node == "action":
          print(f"Tool Used: {values['messages'][0].name}")
        pprint.pprint(values["messages"])
        if node == "agent":
           if values["messages"][0].tool_calls:
              pprint.pprint(values["messages"][0].tool_calls)
        print("\n\n")
  
print(chunk['agent']['messages'][0].content)

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_SlqUFzNvQx8HrcewiEdtHXdI', 'function': {'arguments': '{"query": "QLoRA"}', 'name': 'arxiv'}, 'type': 'function'}, {'id': 'call_iMyxkqyfRsSPDjh60EjvYd1N', 'function': {'arguments': '{"query": "latest Tweet"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 50, 'prompt_tokens': 173, 'total_tokens': 223}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-5acff216-b8b3-4149-9e40-1dba734858a1-0', tool_calls=[{'name': 'arxiv', 'args': {'query': 'QLoRA'}, 'id': 'call_SlqUFzNvQx8HrcewiEdtHXdI', 'type': 'tool_call'}, {'name': 'duckduckgo_search', 'args': {'query': 'latest Tweet'}, 'id': 'call_iMyxkqyfRsSPDjh60EjvYd1N', 'type': 'tool_call'}], usage_metadata={'input_tokens': 173, 'output_tokens': 50, 'total_tokens': 223})]
[{'args': {

####🏗️ Activity #2:

Please write out the steps the agent took to arrive at the correct answer.

## ACTIVITY #2:

1.  STEP 1

-   Agent code appends to the State with an AIMessage containing kwarg tool_calls to call the Arxiv tool using the keyword QLoRa.
Contrary to my expectation, however, I also found that there were tool calls to DuckDuckGo in step 1 - please see the output of the previous cell!

2.  STEP 2

-   The calls to the two tools are sent to the Tool Node ie the Action node where they are executed and the ToolMessages are appended to the sequence of messages.  The action node then passes the full set of messages back to the Agent node.

3.  STEP 3

-   The Agent node receives the full set of messages (latest are the ToolMessages).  This time the agent node formulates very specific search queries, looking for the latest tweet by each of the authors of the paper.

4.  STEP 4

-   The action node executes the searches and appends four more ToolMessages to the sequence of messages.  The full set is again passed back to the Agent node.

5.  STEP 5

-   The Agent node compiles a response to the user, as it has all the information needed.  No Tool Call is made in this step, so the cycle terminates at this point with an AIMessage written to the sequence of messages (this is the response to the user question).

## Part 1: LangSmith Evaluator

### Pre-processing for LangSmith

To do a little bit more preprocessing, let's wrap our LangGraph agent in a simple chain.

In [23]:
def convert_inputs(input_object):
  return {"messages" : [HumanMessage(content=input_object["question"])]}

def parse_output(input_state):
  return input_state["messages"][-1].content

agent_chain = convert_inputs | compiled_graph | parse_output

In [24]:
agent_chain.invoke({"question" : "What is RAG?"})

"RAG stands for Retrieval-Augmented Generation. It is a technique used in natural language processing (NLP) and machine learning to improve the performance of language models by combining retrieval-based methods with generative models. Here's a brief overview of how it works:\n\n1. **Retrieval**: In the first step, the system retrieves relevant documents or pieces of information from a large corpus or database. This is typically done using a retrieval model, such as BM25 or a dense retrieval model like DPR (Dense Passage Retrieval).\n\n2. **Augmentation**: The retrieved documents are then used to augment the input to the generative model. This means that the generative model has access to additional context or information that can help it produce more accurate and relevant responses.\n\n3. **Generation**: Finally, the generative model, such as GPT-3 or BERT, uses the augmented input to generate a response. The additional context provided by the retrieved documents helps the model gener

### Task 1: Creating An Evaluation Dataset

Just as we saw last week, we'll want to create a dataset to test our Agent's ability to answer questions.

In order to do this - we'll want to provide some questions and some answers. Let's look at how we can create such a dataset below.

```python
questions = [
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]
```

####🏗️ Activity #3:

Please create a dataset in the above format with at least 5 questions.

In [25]:
questions = [
    ### YOUR CODE HERE
    "What optimizer is used in QLoRA?",
    "What data type was created in the QLoRA paper?",
    "What is a Retrieval Augmented Generation system?",
    "Who authored the QLoRA paper?",
    "What is the most popular deep learning framework?",
    "What significant improvements does the LoRA system make?"
]

answers = [
    ### YOUR CODE HERE
    {"must_mention" : ["paged", "optimizer"]},
    {"must_mention" : ["NF4", "NormalFloat"]},
    {"must_mention" : ["ground", "context"]},
    {"must_mention" : ["Tim", "Dettmers"]},
    {"must_mention" : ["PyTorch", "TensorFlow"]},
    {"must_mention" : ["reduce", "parameters"]},
]

Now we can add our dataset to our LangSmith project using the following code which we saw last Thursday!

In [26]:
from langsmith import Client

client = Client()
dataset_name = f"Retrieval Augmented Generation - Evaluation Dataset - {uuid4().hex[0:8]}"

dataset = client.create_dataset(
    dataset_name=dataset_name,
    description="Questions about the QLoRA Paper to Evaluate RAG over the same paper."
)

client.create_examples(
    inputs=[{"question" : q} for q in questions],
    outputs=answers,
    dataset_id=dataset.id,
)

#### ❓ Question #3:

How are the correct answers associated with the questions?

> NOTE: Feel free to indicate if this is problematic or not

## ANSWER #3:

The correct answers are associated with the questions based on their corresponding positions in the `questions` list and the `answers` list.  This is problematic for many reasons.  

First, there is a chance that the orders in the questions and answers lists may change if someone accidentally introduces new question/answer pairs and does not put them in the proper order.  

Second, as a more general principle, there is nothing in the setup that shows this connection between questions and answers!  Far better to set up the data in a more structured manner as a separate effort.

### Task 2: Adding Evaluators

Now we can add a custom evaluator to see if our responses contain the expected information.

We'll be using a fairly naive exact-match process to determine if our response contains specific strings.

In [27]:
from langsmith.evaluation import EvaluationResult, run_evaluator

@run_evaluator
def must_mention(run, example) -> EvaluationResult:
    prediction = run.outputs.get("output") or ""
    required = example.outputs.get("must_mention") or []
    score = all(phrase in prediction for phrase in required)
    return EvaluationResult(key="must_mention", score=score)

#### ❓ Question #4:

What are some ways you could improve this metric as-is?

> NOTE: Alternatively you can suggest where gaps exist in this method.

## ANSWER #4:

The metric is very strict!  It requires that the RAG pipeline response must contain the EXACT strings provided in the `must_mention` lists for each query.  This includes matching on case as well!  

1.  The pipeline should be given credit for output that is semantically close to the right answer even if the words/phrases are different.  Some version of cosine similarity or other closeness metric can be used.  

2.  Also, in the case of questions like author of the paper, the pipeline should be given credit for naming one of the four authors, not necessarily only the lead author.  So, perhaps provide a list of acceptable authors, and give credit if one of them is in the response.  The same idea applies to other questions as well, where the acceptable response might be one of several words in a list.  In other words, don't be extremely stringent - partial credit may be needed!!

3.  Finally, give credit even if the answer does not match with the right `case` e.g., lower case vs camelcase, etc.

Now that we have created our custom evaluator - let's initialize our `RunEvalConfig` with it!

In [28]:
from langchain.smith import RunEvalConfig, run_on_dataset

eval_config = RunEvalConfig(
    custom_evaluators=[must_mention],
)

Task 3: Evaluating

All that is left to do is evaluate our agent's response!

In [30]:
client.run_on_dataset(
    dataset_name=dataset_name,
    llm_or_chain_factory=agent_chain,
    evaluation=eval_config,
    verbose=True,
    project_name=f"RAG Pipeline - Evaluation - {uuid4().hex[0:8]}",
    project_metadata={"version": "1.0.0"},
)

View the evaluation results for project 'RAG Pipeline - Evaluation - 6a240b44' at:
https://smith.langchain.com/o/d6cd6bbe-53a2-5b54-80e6-5a211fe37a3d/datasets/a84bce53-4816-4e81-ad2b-fb1bda58b72d/compare?selectedSessions=de5f105c-a352-4f1f-add0-d0e8177d1a63

View all tests for Dataset Retrieval Augmented Generation - Evaluation Dataset - b51d6bd7 at:
https://smith.langchain.com/o/d6cd6bbe-53a2-5b54-80e6-5a211fe37a3d/datasets/a84bce53-4816-4e81-ad2b-fb1bda58b72d
[------------------------------------------------->] 6/6

,feedback.must_mention,error,execution_time,run_id
count,6,0,6.000000,6
unique,2,0,NaN,6
top,False,NaN,NaN,a86e6c8e-6d4b-4dce-8d0a-1b8c99055d45
freq,4,NaN,NaN,1
mean,NaN,NaN,4.681292,NaN
std,NaN,NaN,1.811985,NaN
min,NaN,NaN,2.503537,NaN
25%,NaN,NaN,4.138632,NaN
50%,NaN,NaN,4.437000,NaN
75%,NaN,NaN,4.597860,NaN


{'project_name': 'RAG Pipeline - Evaluation - 6a240b44',
 'results': {'4870dc51-3a1c-42df-a67a-3628d86399e5': {'input': {'question': 'What optimizer is used in QLoRA?'},
   'feedback': [EvaluationResult(key='must_mention', score=True, value=None, comment=None, correction=None, evaluator_info={}, feedback_config=None, source_run_id=UUID('685c8d42-a9c3-49cb-8798-9fb2cc33657f'), target_run_id=None)],
   'execution_time': 4.649164,
   'run_id': 'a86e6c8e-6d4b-4dce-8d0a-1b8c99055d45',
   'output': 'QLoRA (Quantized Low-Rank Adaptation) uses a combination of optimizers to manage memory efficiently during the fine-tuning of large language models. Specifically, it introduces "paged optimizers" to handle memory spikes. This approach allows QLoRA to backpropagate gradients through a frozen, 4-bit quantized pretrained language model into Low Rank Adapters (LoRA), significantly reducing memory usage without sacrificing performance.',
   'reference': {'must_mention': ['paged', 'optimizer']}},
  '4a

## Part 2: LangGraph with Helpfulness:

### Task 3: Adding Helpfulness Check and "Loop" Limits

Now that we've done evaluation - let's see if we can add an extra step where we review the content we've generated to confirm if it fully answers the user's query!

We're going to make a few key adjustments to account for this:

1. We're going to add an artificial limit on how many "loops" the agent can go through - this will help us to avoid the potential situation where we never exit the loop.
2. We'll add to our existing conditional edge to obtain the behaviour we desire.

First, let's define our state again - we can check the length of the state object, so we don't need additional state for this.

In [31]:
class AgentState(TypedDict):
  messages: Annotated[list, add_messages]

Now we can set our graph up! This process will be almost entirely the same - with the inclusion of one additional node/conditional edge!

####🏗️ Activity #5:

Please write markdown for the following cells to explain what each is doing.

##### YOUR MARKDOWN HERE
### Initialize Graph Object and Set Up Two Nodes

In this step, we initialize a new graph object `graph_with_helpfulness_check`.

We also set up two nodes: `agent` and `action` nodes and the associated functions that will be executed by these nodes.  So, e.g., `agent` will execute the `call_model` function and `action` will run the `tool_node` that provides access to the tools needed.

So far, this is identical to the previous graph we set up earlier.


In [32]:
graph_with_helpfulness_check = StateGraph(AgentState)

graph_with_helpfulness_check.add_node("agent", call_model)
graph_with_helpfulness_check.add_node("action", tool_node)

##### YOUR MARKDOWN HERE

### Set entry point to graph

As before, we set the entry point to the graph as the `agent` node.

In [33]:
graph_with_helpfulness_check.set_entry_point("agent")

##### YOUR MARKDOWN HERE

### Check if the agent should terminate or continue

This cell defines the function that determines if the agent should terminate or continue.

1.  If the last message has a non-empty `tool_calls` kwarg, then the agent is routed to the `action` node.

2.  If not, then:
-   The `agent` will terminate if there are more than 10 Message objects in the AgentState.
-   OR, if the `helpfulness_response` has returned a "Y" ie that the LLM call in this step determines that the final response is helpful as a response to the initial query.

3.  If the `agent` has not accumulated more than 10 message objects in the AgentState, and if the response is not deemed to be helpful, the agent will continue.

In [34]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser

def tool_call_or_helpful(state):
  last_message = state["messages"][-1]

  if last_message.tool_calls:
    return "action"

  initial_query = state["messages"][0]
  final_response = state["messages"][-1]

  if len(state["messages"]) > 10:
    return "END"

  prompt_template = """\
  Given an initial query and a final response, determine if the final response is extremely helpful or not. Please indicate helpfulness with a 'Y' and unhelpfulness as an 'N'.

  Initial Query:
  {initial_query}

  Final Response:
  {final_response}"""

  prompt_template = PromptTemplate.from_template(prompt_template)

  helpfulness_check_model = ChatOpenAI(model="gpt-4")

  helpfulness_chain = prompt_template | helpfulness_check_model | StrOutputParser()

  helpfulness_response = helpfulness_chain.invoke({"initial_query" : initial_query.content, "final_response" : final_response.content})

  if "Y" in helpfulness_response:
    return "end"
  else:
    return "continue"

####🏗️ Activity #4:

Please write what is happening in our `tool_call_or_helpful` function!

1.  Get the last ie most recent message from the state.  If this message has a non-empty tool-call, then execute the tool call.

2.  If the last message is not a tool-call, then check if either of the following conditions is met:
-   has the length of the list of messages in the agent state exceeded 10?  OR
-   does the LLM call in this step deem the latest response to be helpful given the initial query?
If the answer to either one of these questions is a 'yes', then the agent will terminate.

3.  If neither condition in 2. is met, then the agent will continue.

##### YOUR MARKDOWN HERE

### Add conditional Edges

This step sets up conditional edges from the `agent` node to one of three alternatives:
1.  If tool_call_or_helpful function returns `action`, then loop to `action` node
2.  If tool_call_or_helpful function returns `end`, then terminate the agent.
3.  If tool_call_or_helpful function returns `continue`, then loop back to `agent` node.

In [35]:
graph_with_helpfulness_check.add_conditional_edges(
    "agent",
    tool_call_or_helpful,
    {
        "continue" : "agent",
        "action" : "action",
        "end" : END
    }
)

##### YOUR MARKDOWN HERE

### Add Edge from Action to Agent

This step adds an edge from action back to agent.  So, this means that every time the action node is executed, the agent will return to the agent node.

In [36]:
graph_with_helpfulness_check.add_edge("action", "agent")

##### YOUR MARKDOWN HERE

## Compile Graph

This step compiles the graph.  After this step is completed, the agent is ready to execute.

In [37]:
agent_with_helpfulness_check = graph_with_helpfulness_check.compile()

##### YOUR MARKDOWN HERE

### Run the Agent!

1.  Run the agent in async streaming mode.
2.  Print out the results as the agent is executing

SEE RESULTS BELOW!!

1.  On the first call, there are three search terms extracted and each of them is set up with a tool call to DuckDuckGoSearch.
2.  The tool calls return relevant content for each question.
3.  The final AI message has responses to all three questions posed in the input.

In [40]:
inputs = {"messages" : [HumanMessage(content="Related to machine learning, what is LoRA? Also, who is Tim Dettmers? Also, what is Attention?")]}

async for chunk in agent_with_helpfulness_check.astream(inputs, stream_mode="updates"):
    for node, values in chunk.items():
        print(f"Receiving update from node: '{node}'")
        print(values["messages"])
        print("\n\n")

print(chunk['agent']['messages'][0].content)

Receiving update from node: 'agent'
[AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_zd6hh34mqdpKx5k4T6iQlJXY', 'function': {'arguments': '{"query": "LoRA machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_P1LNZqm0wydBVj0yeCopx4RK', 'function': {'arguments': '{"query": "Tim Dettmers"}', 'name': 'duckduckgo_search'}, 'type': 'function'}, {'id': 'call_1qtQMLUZLSeDBWOmFhM4xEvE', 'function': {'arguments': '{"query": "Attention in machine learning"}', 'name': 'duckduckgo_search'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 76, 'prompt_tokens': 171, 'total_tokens': 247}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_157b3831f5', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-427d3690-d09b-4118-beb2-9aeb542ef1f2-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': 'LoRA machine learning'}, 'id': 'call_zd6hh34mqdpKx5k4T6iQlJXY', 'type': 'tool_call'}, 

### Task 4: LangGraph for the "Patterns" of GenAI

Let's ask our system about the 4 patterns of Generative AI:

1. Prompt Engineering
2. RAG
3. Fine-tuning
4. Agents

In [41]:
patterns = ["prompt engineering", "RAG", "fine-tuning", "LLM-based agents"]

In [42]:
for pattern in patterns:
  what_is_string = f"What is {pattern} and when did it break onto the scene??"
  inputs = {"messages" : [HumanMessage(content=what_is_string)]}
  messages = agent_with_helpfulness_check.invoke(inputs)
  print(messages["messages"][-1].content)
  print("\n\n")

Prompt engineering is a concept primarily associated with the field of artificial intelligence, particularly in the context of natural language processing (NLP) and large language models like GPT-3. It involves the design and crafting of prompts (input text) to elicit desired responses from AI models. The goal is to optimize the input to get the most accurate, relevant, or useful output from the model.

### Key Aspects of Prompt Engineering:
1. **Crafting Effective Prompts**: Designing questions or statements that guide the AI to produce the desired type of response.
2. **Iterative Testing**: Continuously refining prompts based on the responses received to improve the quality of the output.
3. **Understanding Model Behavior**: Gaining insights into how the model interprets different types of input to better predict and control its responses.

### When Did It Break Onto the Scene?
Prompt engineering became more prominent with the advent of large-scale language models like OpenAI's GPT-3